In [1]:
import os
import codecs
import pprint
import pymorphy2
import datetime
import psycopg2

from nltk import FreqDist
from nltk.util import ngrams
from nltk.corpus import stopwords

In [2]:
def clearDB(fileID):
    sql = "DELETE FROM terms WHERE doc_code = %s"
    conn = psycopg2.connect(host="localhost", database="terms", user="postgres", password="admin")

    cur = conn.cursor()
    cur.execute(sql, (fileID, ))
    conn.commit()
    cur.close()
    conn.close()

In [3]:
def prepareData(fileName):
    data = []
    stop_words = getStopWords('russian')
    file = codecs.open(dirName + "/" + fileName, mode="r", encoding="utf-8-sig")
    for line in file:       # читаем по строкам
        # убираем знаки пунктуации
        line = line.replace('(', ' ')
        line = line.replace(')', ' ')
        line = line.replace('.', ' ')
        line = line.replace(',', ' ')
        line = line.replace(':', ' ')
        line = line.replace(';', ' ')
        line = line.replace('№', ' ')
        line = line.replace('#', ' ')
        line = line.replace('"', ' ')
        line = line.replace('«', ' ')
        line = line.replace('«', ' ')
        line = line.replace('—', '-')
        line = line.replace('–', '-')

        # переводим все слова в нижний регистр
        line = line.lower()
        r = line.split()    # массив слов

        # убираем стоп-слова
        r = [i for i in r if (i not in stop_words)]

        if len(r) > 0:
            data.append(r)

    return data

In [4]:
def getStopWords(lang):
    stop_words = stopwords.words(lang)
    stop_words.extend(
    ['всё', 'все',
    'своей', 'своя', 'свою', 'своими',
    'которая', 'который', 'которые', 'которое',
    'иной', 'иная', 'иное', 'иные', 'иных', 'иными',
    'другой', 'другая', 'другие', 'другое',
    'соответствующий',
    'что',
    'это', 'этот', 'эти', 'этих', 'эта', 'этому',
    'тот', 'те', 'тех', 'та',
    'так', 'также',
    'вот',
    'быть',
    'как',
    'в',
    '—', '–', '-',
    'к',
    'на',
    '...',
    'декабрь',    'январь',    'февраль',
    'март',    'апрель',    'май',
    'июнь',    'июль',    'август',
    'сентябрь',    'октябрь',    'ноябрь',
    'казахстан', 'республика',
    '1', '2', '3', '4', '5', '6', '7', '8', '9', '0'])
    
    return stop_words

In [5]:
def prepareFreqDistNoun(data):
    fdist = FreqDist()

    # считаем частоту существительных (переводим в единственное число и именительный падеж)
    # Термин состоит из СУЩЕСТВИТЕЛЬНОГО в ИМЕНИТЕЛЬНОМ падеже
    for line in data:
        for w in line:
            if (w[0:1] == '-'):
                w = w[1:]

            try:
                m = morph.parse(w)[0]
                if m.tag.POS == 'NOUN':
                    snw = morph.parse(w)[0].inflect({'sing', 'nomn'})
                    fdist[snw.word] += 1
            except:
##                print("Error:")
##                pprint.pprint(w)
                pass
    return fdist

In [6]:
def writeFreqDist2DB(freq, fileID):
    sql = "INSERT INTO terms (term, doc_code, cnt) VALUES (%s, %s, %s)"
    conn = psycopg2.connect(host="localhost",database="terms", user="postgres", password="admin")

    cur = conn.cursor()

    for w in freq.most_common():
        cur.execute("INSERT INTO terms (term, doc_code, cnt) VALUES (%s, %s, %s)", ("qwerty", fileID.upper(), "333"))
        #cur.execute(sql, (w[0], fileID.upper(), w[1]))
        conn.commit()

    cur.close()
    conn.close()

In [7]:
#import nltk
#nltk.download('stopwords')
dirName = "corpus"
listdir = os.listdir(dirName)

for fileName in listdir:           # каждый файл в директории
    fileID = fileName.split('.')[0]
    print(fileID)
    clearDB(fileID)
    data = prepareData(fileName)
    
    dict1w = FreqDist()
    dict1w.update(prepareFreqDistNoun(data))
    writeFreqDist2DB(dict1w, fileID)    


k030000442_
k030000481_
k070000212_
k090000193_
k1400000235
k1500000375
k1500000414
k1700000125
k940001000_
k990000409_
p1000001202
z000000053_
z010000242_
z010000245_
z020000332_
z030000476_
z040000567_
z040000588_
z040000603_
z060000175_
z090000165_
z1100000469
z1200000020
z1200000532
z1200000541
z1400000188
z1400000194
z1400000202
z1500000434
z1600000480
z1600000486
z1800000183
z920002900_
z940007000_
z960000006_
